# PySpark Practice

## Load libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

## Create dataframe

In [ ]:
mydata = spark.read.format("csv").option("header", "true").load("original.csv")


In [ ]:
mydata.show()

## Create a new city row where null entries are replaced with "unknown"

In [ ]:
mydata2 = mydata.withColumn("clean_city", when(mydata.City.isNull(),"unknown").otherwise(mydata.City))

In [ ]:
mydata2.show()

## Filter out all rows where the job title is null

In [ ]:
mydata2 = mydata2.filter(mydata2.JobTitle.isNotNull())

In [ ]:
mydata2.show()

## Change salary column from strings to float

In [ ]:
mydata2 = mydata2.withColumn("clean_salary", mydata2.Salary.substr(2, 100).cast("float"))
# starting at column 2 means the dollar sign doesn't get processed

## Replace null salaries with the mean value

In [ ]:
mean = mydata2.groupBy().avg("clean_salary").take(1)[0][0]
print(mean)

In [ ]:
from pyspark.sql.functions import lit
mydata2 = mydata2.withColumn("new_salary", when(mydata2.clean_salary.isNull(), lit(mean)).otherwise(mydata2.clean_salary))
# If new_salary is null, insert the literal result of the mean, otherwise stay as clean salary
# safe to do whether you know for sure there are any null values or not. Important if the data is going to be regularly updated

In [ ]:
mydata2.show()

## Remove nulls for Latitude

In [ ]:
import numpy as np
latitudes = mydata2.select("Latitude")

In [ ]:
latitudes = latitudes.filter(latitudes.Latitude.isNotNull())

In [ ]:
latitudes.show()

## Convert Latitutde from string to float

In [ ]:
latitudes = latitudes.withColumn("latitude2", latitudes.Latitude.cast("float")).select("latitude2")

## Replace Latitude nulls with median average

In [ ]:
median = np.median(latitudes.collect())

In [ ]:
mydata2 = mydata2.withColumn("lat", when(mydata2.Latitude.isNull(), lit(median)).otherwise(mydata2["Latitude"]))

In [ ]:
mydata2.show()